# Topic Modelling using LDA

> ☝️Before moving on with this notebook, ensure that you have:
- The following packages installed into your conda environment that runs this notebook: `pip install gensim spacy pyLDAvis matplotlib numpy pandas nltk`

**Overview**: In this notebook, we will explore topic modelling using Latent Dirichlet Allocation (LDA) on a corpus of Geological Survey of Western Australia reports.

⚠️This notebook takes approximately 50 minutes to execute all cells. The visualisations of the topic models is the culprit.

## Table of Contents
1. [Loading GSWA Corpus](#load_corpus)
2. [Corpus Pre-processing](#preprocessing)
3. [Training LDA Model](#training)
4. [Interactive Visualisation](#visualisation)
5. [Additional Content: LDA Model Hyperparameter Tuning](#tuning)

### Import Dependencies
- [re](https://docs.python.org/3/library/re.html) - library that we will use for performing regular expressions
- [numpy](https://numpy.org/) - library that we will use for helping visualise results
- [pandas](https://pandas.pydata.org/) - library that we will use to help handling our corpus
- [gensim](https://radimrehurek.com/gensim/) - library that we will use to perform topic modelling
- [spacy](https://spacy.io/) - library that we will use to perform lemmatization
- [nltk](https://www.nltk.org/) - library that we will use to pre-process our corpus
- [pyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html) - library that we will use to interactively visualise results

In [1]:
import re
from pprint import pprint
import logging
import warnings
import zipfile
import json
from typing import List

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

### Set-up notebook environment and helper functions

In [2]:
# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\22917746\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Import the GSWA dataset <a name="load_corpus"></a>

The 20-Newsgroups dataset contains about 6k data posts from 1,135 GSWA reports. 

The dataset can be imported using `pandas.read_json` and we can see the dataset has 3 columns. The `target-name` is the category or topics that the news is manually assigned to. 

We choose to use this dataset because it has pre-assigned categories according to topics, so our clustering results can be readily compared. In real world data, this is not always easily available. 

In [12]:
corpus_path = r"../data/wamex_xml.zip"

In [13]:
corpus = list()
with zipfile.ZipFile(corpus_path, "r") as z:
    for filename in z.namelist():
        with z.open(filename) as f:
            # load the json file
            # The resulting `content` is a list
            content = json.loads(f.read()) 
            # Convert content to a string   
            content = "".join(content)
            # Add to the data list
            corpus.append(content)

In [14]:
pprint(corpus[:1])

['Combined Reporting Application Pending Status: Sheet 1:250 000: Kalgoorlie '
 '(SH 51-09) Sheet 1:100 000: Kalgoorlie (3136) Project Operator: Cazaly '
 'Resources Limited Author: M Watts Date: September 2006 Distribution: '
 '1.Department of Industry and Resources 2.Cazaly Resources Limited Cazaly '
 'Resources Limited September 2006 2 Figure 2: Castle Hill Project Tenement '
 'Location Plan, 1: 75 000 scale 6 Figure 3: Regional Geology with Exploration '
 'Index Map, 1:75 000 scale 8Cazaly Resources Limited September 2006 3 1.0 '
 'SUMMARY The Combined Mineral Exploration Report on the Castle Hill Project '
 'details exploration activities undertaken by Cazaly Resources Limited during '
 'the reporting period from 7 July 2005 to 6 July 2006.The reporting group '
 'known as Castle Hill comprises nine granted prospecting licences.The project '
 'area covers some 1250 hectares.Cazaly is the registered holder of the Castle '
 'Hill Project tenements.Cazalys exploration activities durin

## Pre-processing <a name="preprocessing"></a>

Before performing topic modelling, we need to pre-process our corpus. Here we will remove unnecessary information using regular expressions via `re`.

In [15]:
# Create set of English stopwords and include domain-specific words 
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [16]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [17]:
def lemmatize_doc(tokenized_doc: List[str], allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[str]:
    '''Document lemmatization using SpaCy
        
        Ref:
        - https://spacy.io/api/annotation
    '''
    
    spacy_doc = nlp(" ".join(tokenized_doc))
    doc_out = []
    for token in spacy_doc:
        if token.pos_ in allowed_postags:
            doc_out.append(token.lemma_)
        else:
            doc_out.append(token.text)
    return doc_out

In [18]:
def preprocess_and_tokenize_corpus(corpus: List[str],
                      stopwords: List[str],
                      allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[List[str]]:
    '''
    Preprocesses a corpus of texts. 
    '''
    
    preprocessed_corpus = []
    for doc in corpus:
        if len(" ".join(doc))==0 or len(" ".join(doc))>1000000:
            pass
        else:
            # Remove Figure No.'s
            doc = re.sub('Figure\s+\d+', '', doc)
            # Remove all numbers
            doc = re.sub('\d+', '', doc)
            # Remove new line characters
            doc = re.sub('\s+', ' ', doc)
            # Remove distracting single quotes
            doc = re.sub("\'", '', doc)
            # Run Gensim's simple_preprocesser - this tokenizes the document
            doc = simple_preprocess(str(doc), deacc=True)
            # Remove stopwords
            doc = [word for word in doc  if word not in stopwords]
            # Lemmatize
            doc = lemmatize_doc(doc)
            
            preprocessed_corpus.append(doc)
    return preprocessed_corpus

In [19]:
# Pre-process and tokenize corpus
tokenized_corpus = preprocess_and_tokenize_corpus(corpus=corpus, stopwords=stop_words)

In [20]:
# View part of pre-processed corpus
pprint(tokenized_corpus[:1])

[['combined',
  'reporting',
  'application',
  'pende',
  'status',
  'sheet',
  'kalgoorlie',
  'sh',
  'sheet',
  'kalgoorlie',
  'project',
  'operator',
  'cazaly',
  'resources',
  'limited',
  'author',
  'watts',
  'date',
  'september',
  'distribution',
  'department',
  'industry',
  'resource',
  'cazaly',
  'resource',
  'limited',
  'cazaly',
  'resource',
  'limit',
  'september',
  'castle',
  'hill',
  'project',
  'tenement',
  'location',
  'plan',
  'scale',
  'regional',
  'geology',
  'exploration',
  'index',
  'map',
  'scale',
  'cazaly',
  'resource',
  'limit',
  'september',
  'summary',
  'combine',
  'mineral',
  'exploration',
  'report',
  'castle',
  'hill',
  'project',
  'detail',
  'exploration',
  'activity',
  'undertake',
  'cazaly',
  'resource',
  'limited',
  'reporting',
  'period',
  'july',
  'july',
  'report',
  'group',
  'know',
  'castle',
  'hill',
  'comprise',
  'nine',
  'grant',
  'prospect',
  'licence',
  'project',
  'area',
  '

### Creating bigram and trigrams - Gensim’s Phrases model

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In this notebook, we focus on building a bigram model. However the code that is required to build a trigram model is made available (it just needs to be converted to a code cell).

In [21]:
bigram = gensim.models.Phrases(tokenized_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.

Faster way to get a sentence clubbed as a trigram/bigram

In [22]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

View example of bigram model

In [23]:
# See example of bigram model applied to pre-processed corpus
pprint(bigram_mod[tokenized_corpus[0]])

['combined',
 'reporting',
 'application',
 'pende',
 'status',
 'sheet',
 'kalgoorlie',
 'sh',
 'sheet',
 'kalgoorlie',
 'project',
 'operator',
 'cazaly_resources',
 'limited',
 'author',
 'watts',
 'date',
 'september',
 'distribution',
 'department_industry',
 'resource',
 'cazaly',
 'resource',
 'limited',
 'cazaly',
 'resource',
 'limit',
 'september',
 'castle_hill',
 'project',
 'tenement',
 'location',
 'plan',
 'scale',
 'regional',
 'geology',
 'exploration',
 'index',
 'map',
 'scale',
 'cazaly',
 'resource',
 'limit',
 'september',
 'summary',
 'combine',
 'mineral',
 'exploration',
 'report',
 'castle_hill',
 'project',
 'detail',
 'exploration',
 'activity',
 'undertake',
 'cazaly',
 'resource',
 'limited',
 'reporting_period',
 'july',
 'july',
 'report',
 'group',
 'know',
 'castle_hill',
 'comprise',
 'nine',
 'grant',
 'prospect',
 'licence',
 'project',
 'area',
 'cover',
 'hectares_cazaly',
 'registered',
 'holder',
 'castle_hill',
 'project',
 'tenements',
 'cazal

Convert tokenized corpus into bi-gram phrases - this will allow the topic model to capture domain specific phrases in addition to individual words

In [24]:
bigram_corpus = [bigram_mod[tokenized_doc] for tokenized_doc in tokenized_corpus]

To perform topic modelling with LDA we need to convert the corpus of pre-processed texts into a bag-of-words (bow) representation with count frequencies.

In [27]:
# Create a dictionary that maps each word to a unique id
id2word = corpora.Dictionary(bigram_corpus)

In [29]:
id2word[0]

'accordingly'

Convert the bigram corpus into a [bag-of-words (bow) representation](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

In [30]:
bow_corpus = [id2word.doc2bow(text) for text in bigram_corpus]

In [32]:
# Review a single document in bow format; (index, count frequency) 
print(bow_corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 8), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 17), (13, 3), (14, 1), (15, 1), (16, 4), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 13), (26, 10), (27, 3), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 2), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 4), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 2), (54, 3), (55, 1), (56, 1), (57, 3), (58, 1), (59, 3), (60, 1), (61, 4), (62, 1), (63, 1), (64, 3), (65, 1), (66, 1), (67, 13), (68, 1), (69, 1), (70, 1), (71, 3), (72, 1), (73, 2), (74, 2), (75, 2), (76, 3), (77, 1), (78, 1), (79, 1), (80, 5), (81, 4), (82, 3), (83, 3), (84, 1), (85, 4), (86, 1), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 4), (105, 7), (106, 1), (107, 1), (108, 1), (109, 2), (110

In [34]:
# View the corresponding corpus with bigrams
print(bigram_corpus[0])

['combined', 'reporting', 'application', 'pende', 'status', 'sheet', 'kalgoorlie', 'sh', 'sheet', 'kalgoorlie', 'project', 'operator', 'cazaly_resources', 'limited', 'author', 'watts', 'date', 'september', 'distribution', 'department_industry', 'resource', 'cazaly', 'resource', 'limited', 'cazaly', 'resource', 'limit', 'september', 'castle_hill', 'project', 'tenement', 'location', 'plan', 'scale', 'regional', 'geology', 'exploration', 'index', 'map', 'scale', 'cazaly', 'resource', 'limit', 'september', 'summary', 'combine', 'mineral', 'exploration', 'report', 'castle_hill', 'project', 'detail', 'exploration', 'activity', 'undertake', 'cazaly', 'resource', 'limited', 'reporting_period', 'july', 'july', 'report', 'group', 'know', 'castle_hill', 'comprise', 'nine', 'grant', 'prospect', 'licence', 'project', 'area', 'cover', 'hectares_cazaly', 'registered', 'holder', 'castle_hill', 'project', 'tenements', 'cazalys', 'exploration', 'activity', 'report', 'period', 'direct_towards', 'acquisit

In [35]:
# Sort documents by term frequency e.g. most frequency first
sorted_by_value = sorted(bow_corpus[0], key=lambda kv: kv[1], reverse=True)
print(sorted_by_value)

[(162, 27), (12, 17), (25, 13), (67, 13), (26, 10), (174, 10), (208, 9), (3, 8), (105, 7), (178, 7), (136, 6), (177, 6), (216, 6), (229, 6), (80, 5), (117, 5), (124, 5), (159, 5), (183, 5), (16, 4), (46, 4), (61, 4), (81, 4), (85, 4), (104, 4), (115, 4), (116, 4), (170, 4), (175, 4), (211, 4), (1, 3), (13, 3), (27, 3), (28, 3), (35, 3), (52, 3), (54, 3), (57, 3), (59, 3), (64, 3), (71, 3), (76, 3), (82, 3), (83, 3), (111, 3), (129, 3), (134, 3), (140, 3), (151, 3), (154, 3), (160, 3), (166, 3), (173, 3), (179, 3), (184, 3), (189, 3), (194, 3), (215, 3), (235, 3), (5, 2), (6, 2), (10, 2), (29, 2), (36, 2), (37, 2), (42, 2), (44, 2), (47, 2), (53, 2), (73, 2), (74, 2), (75, 2), (90, 2), (94, 2), (109, 2), (114, 2), (123, 2), (125, 2), (127, 2), (128, 2), (137, 2), (150, 2), (158, 2), (163, 2), (176, 2), (180, 2), (187, 2), (188, 2), (191, 2), (195, 2), (200, 2), (218, 2), (219, 2), (0, 1), (2, 1), (4, 1), (7, 1), (8, 1), (9, 1), (11, 1), (14, 1), (15, 1), (17, 1), (18, 1), (19, 1), (20, 

In [37]:
# View a human readable format of corpus (term-frequency)
[(id2word[id], freq) for id, freq in sorted_by_value] 

[('project', 27),
 ('area', 17),
 ('castle_hill', 13),
 ('exploration', 13),
 ('cazaly', 10),
 ('report', 10),
 ('tenement', 9),
 ('activity', 8),
 ('kalgoorlie', 7),
 ('review', 7),
 ('nickel', 6),
 ('resource', 6),
 ('ultramafic', 6),
 ('within', 6),
 ('geology', 5),
 ('limited', 5),
 ('mafic', 5),
 ('potential', 5),
 ('september', 5),
 ('available', 4),
 ('coolgardie', 4),
 ('domain', 4),
 ('gold', 4),
 ('group', 4),
 ('july', 4),
 ('lie', 4),
 ('limit', 4),
 ('regional', 4),
 ('reporting', 4),
 ('terrane', 4),
 ('acquisition', 3),
 ('assess', 3),
 ('cazaly_resources', 3),
 ('cazalys', 3),
 ('combine', 3),
 ('dataset', 3),
 ('datum', 3),
 ('detail', 3),
 ('direct_towards', 3),
 ('eastern_goldfields', 3),
 ('fault', 3),
 ('follow', 3),
 ('grant', 3),
 ('greenstone', 3),
 ('kunanalle', 3),
 ('mineral', 3),
 ('nearby', 3),
 ('north', 3),
 ('part', 3),
 ('period', 3),
 ('previous', 3),
 ('rate', 3),
 ('relate', 3),
 ('rock', 3),
 ('sequence', 3),
 ('sheet', 3),
 ('structural', 3),
 ('tr

## Train LDA Model <a name="training"></a>

<img src="https://miro.medium.com/max/800/1*pZo_IcxW1GVuH2vQKdoIMQ.jpeg" alt="LDA example" style="width:600px;"/>

Here we train a LDA topic mode on our bag-of-words corpus. We initially set the `number of topics` we believe are in the corpus to 10. Only a single `pass` is performed through the corpus, but this could be increased to get better results. However, it is computationally expensive.

In [38]:
%timeit
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,   # To get an improved LDA model, this can be increased
                                           alpha='auto',
                                           per_word_topics=True)

Here we show the topics elicited from the LDA model and the top keywords in the 10 topics. This shows the weighted importance of each word in each topic.

In [39]:
pprint(lda_model.print_topics())

[(0,
  '0.017*"area" + 0.015*"exploration" + 0.014*"tenement" + 0.014*"project" + '
  '0.013*"report" + 0.011*"australia" + 0.010*"western" + 0.008*"within" + '
  '0.006*"deposit" + 0.006*"ltd"'),
 (1,
  '0.015*"coal" + 0.013*"sandstone" + 0.012*"report" + 0.009*"exploration" + '
  '0.008*"drill" + 0.007*"basin" + 0.006*"sample" + 0.006*"survey" + '
  '0.006*"mineral" + 0.005*"irwin_river"'),
 (2,
  '0.013*"resource" + 0.010*"use" + 0.010*"grade" + 0.010*"datum" + '
  '0.008*"pit" + 0.008*"model" + 0.008*"report" + 0.008*"drill" + 0.007*"hole" '
  '+ 0.007*"sample"'),
 (3,
  '0.015*"gold" + 0.014*"area" + 0.010*"tenement" + 0.010*"sample" + '
  '0.010*"project" + 0.009*"exploration" + 0.008*"report" + 0.008*"hole" + '
  '0.008*"drilling" + 0.007*"drill"'),
 (4,
  '0.061*"survey" + 0.017*"intrusion" + 0.016*"kimberley" + 0.016*"datum" + '
  '0.012*"line" + 0.008*"em" + 0.007*"hall_creek" + 0.007*"station" + '
  '0.007*"model" + 0.006*"area"'),
 (5,
  '0.025*"survey" + 0.013*"site" + 0.0

In [40]:
doc_lda = lda_model[bow_corpus]

To evaluate the quality of our topic model, we can compute its `perplexity` and `coherence`.

In [42]:
# Compute Perplexity - lower is better.
print(f'Perplexity: {lda_model.log_perplexity(bow_corpus)}')

Perplexity: -8.55996523766831


In [43]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=bigram_corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.45146853151923994


## Interactive LDA Visualisation <a name="visualisation"></a> 

#### Wait, what am I looking at again?

There are a lot of moving parts in the visualization. Here's a brief summary:

On the left, there is a plot of the "distance" between all of the topics (labeled as the Intertopic Distance Map)
The plot is rendered in two dimensions according a multidimensional scaling (MDS) algorithm. Topics that are generally similar should be appear close together on the plot, while dissimilar topics should appear far apart.

The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.

An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.

On the right, there is a bar chart showing top terms.
When no topic is selected in the plot on the left, the bar chart shows the top most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.

When a particular topic is selected, the bar chart changes to show the top most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter λλ, which can be adjusted with a slider above the bar chart.

Setting the λλ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.

Setting λλ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic — i.e., terms that occur only in this topic, and do not occur in other topics.

Setting λλ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.

Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found here. Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's LdaMulticore object and pyLDAvis' visualization, you have to dig through the terms manually.

#### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:

Better understanding and interpreting individual topics, and Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the λλ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

#### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% Topic A, 20% Topic B, 20% Topic C, and 10% Topic D.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:

Using spaCy to remove punctuation and lemmatize the text

- Applying our first-order phrase model to join word pairs
- Applying our second-order phrase model to join longer phrases
- Removing stopwords
- Creating a bag-of-words representation
- Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The lda_description(...) function will perform all these steps for us, including printing the resulting topical description of the input text.

##### Acknowledgement: https://www.kaggle.com/navinch/interesting-visualizations-lda-word2vec

### Visualize the topics

In [44]:
pyLDAvis.enable_notebook()

In [46]:
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.010243 -0.025095       1        1  25.512289
0     -0.015769  0.015296       2        1  15.510426
6      0.046251 -0.000992       3        1  14.655152
2     -0.034741  0.051800       4        1  12.649196
9      0.066991 -0.081405       5        1  10.696554
7      0.228366  0.008372       6        1   6.989476
8     -0.051802  0.026277       7        1   4.395954
4     -0.118022 -0.151663       8        1   3.376314
5     -0.085870  0.086224       9        1   3.187277
1     -0.045645  0.071186      10        1   3.027361, topic_info=           Term          Freq         Total Category  logprob  loglift
1097     survey   9664.000000   9664.000000  Default  30.0000  30.0000
54        datum   7743.000000   7743.000000  Default  29.0000  29.0000
924      quartz   5305.000000   5305.000000  Default  28.0000  28.0000
672   intrusion   2485.000000   2485.000000  Default  27.0000  27.0000
179        rock  10540.000000  10540.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
357        chip    177.292235   2050.220574  Topic10  -5.6010   1.0496
54        datum    195.754549   7743.401700  Topic10  -5.5019  -0.1803
162     project    189.272696  11088.946749  Topic10  -5.5356  -0.5730
12         area    189.056280  16003.531504  Topic10  -5.5367  -0.9410
482    drilling    177.604869   7742.371393  Topic10  -5.5992  -0.2774

[851 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
5100       1  0.052459  aboriginal
5100       2  0.931919  aboriginal
5100       5  0.003086  aboriginal
5100       8  0.006172  aboriginal
2513       1  0.022994      acacia
...      ...       ...         ...
235        7  0.017290        zone
235        8  0.014926        zone
235        9  0.013309        zone
235       10  0.006468        zone
11370      3  0.897219     zuleika

[3594 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 7, 3, 10, 8, 9, 5, 6, 2])

## Additional Content: Tuning the number of topics (hyperparameter tuning) <a name="tuning"></a>

Here we explore how to find the best number of topics for our LDA model. This involves iteratively computing coherence values and finding an optimal topic distribution over a given number of topics.

⚠️ Computing coherence values can take a very long time to run.

Here we can visualise coherence scores as a function of the number of topics